In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"],1)
application_df

C:\Users\rghaf\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
application_type = application_df.APPLICATION_TYPE.value_counts()
application_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Determine which values to replace if counts are less than 100?
replace_application = list(application_type[application_type < 100].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification = application_df.CLASSIFICATION.value_counts()
classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification[classification < 777].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[app_cat].nunique()

APPLICATION_TYPE          10
AFFILIATION                6
CLASSIFICATION             7
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

C:\Users\rghaf\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(app_cat,1)
application_df.head()

C:\Users\rghaf\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                4140      
                                                                 
 dense_1 (Dense)             (None, 40)                3640      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 7,821
Trainable params: 7,821
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 678us/step - loss: 0.5721 - accuracy: 0.7185
Epoch 2/50
804/804 [==============================] - 1s 714us/step - loss: 0.5555 - accuracy: 0.7278
Epoch 3/50
804/804 [==============================] - 1s 675us/step - loss: 0.5525 - accuracy: 0.7285
Epoch 4/50
804/804 [==============================] - 1s 683us/step - loss: 0.5513 - accuracy: 0.7304
Epoch 5/50
753/804 [===========================>..] - ETA: 0s - loss: 0.5491 - accuracy: 0.7314
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 682us/step - loss: 0.5497 - accuracy: 0.7306
Epoch 6/50
804/804 [==============================] - 1s 763us/step - loss: 0.5488 - accuracy: 0.7306
Epoch 7/50
804/804 [==============================] - 1s 668us/step - loss: 0.5471 - accuracy: 0.7314
Epoch 8/50
804/804 [==============================] - 1s 665us/step - loss: 0.5473 - accuracy: 0.7333
Epoch 9/50
804/804 [================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5531 - accuracy: 0.7301 - 212ms/epoch - 792us/step
Loss: 0.553063154220581, Accuracy: 0.7301457524299622


In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               4600      
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9,701
Trainable params: 9,701
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [22]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 700us/step - loss: 0.5684 - accuracy: 0.7212
Epoch 2/50
804/804 [==============================] - 1s 690us/step - loss: 0.5573 - accuracy: 0.7256
Epoch 3/50
804/804 [==============================] - 1s 676us/step - loss: 0.5541 - accuracy: 0.7270
Epoch 4/50
804/804 [==============================] - 1s 685us/step - loss: 0.5530 - accuracy: 0.7283
Epoch 5/50
765/804 [===========================>..] - ETA: 0s - loss: 0.5518 - accuracy: 0.7281
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 744us/step - loss: 0.5512 - accuracy: 0.7287
Epoch 6/50
804/804 [==============================] - 1s 690us/step - loss: 0.5503 - accuracy: 0.7299
Epoch 7/50
804/804 [==============================] - 1s 691us/step - loss: 0.5499 - accuracy: 0.7314
Epoch 8/50
804/804 [==============================] - 1s 707us/step - loss: 0.5494 - accuracy: 0.7307
Epoch 9/50
804/804 [================

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5521 - accuracy: 0.7287 - 181ms/epoch - 677us/step
Loss: 0.5521187782287598, Accuracy: 0.7287463545799255


In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 70
hidden_nodes_layer3 = 40

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 90)                4140      
                                                                 
 dense_10 (Dense)            (None, 70)                6370      
                                                                 
 dense_11 (Dense)            (None, 40)                2840      
                                                                 
 dense_12 (Dense)            (None, 1)                 41        
                                                                 
Total params: 13,391
Trainable params: 13,391
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [30]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 935us/step - loss: 0.5346 - accuracy: 0.7386
Epoch 2/100
804/804 [==============================] - 1s 820us/step - loss: 0.5346 - accuracy: 0.7385
Epoch 3/100
804/804 [==============================] - 1s 780us/step - loss: 0.5346 - accuracy: 0.7404
Epoch 4/100
804/804 [==============================] - 1s 778us/step - loss: 0.5346 - accuracy: 0.7391
Epoch 5/100
736/804 [==========================>...] - ETA: 0s - loss: 0.5356 - accuracy: 0.7380
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 778us/step - loss: 0.5344 - accuracy: 0.7391
Epoch 6/100
804/804 [==============================] - 1s 758us/step - loss: 0.5342 - accuracy: 0.7393
Epoch 7/100
804/804 [==============================] - 1s 769us/step - loss: 0.5351 - accuracy: 0.7387
Epoch 8/100
804/804 [==============================] - 1s 749us/step - loss: 0.5339 - accuracy: 0.7399
Epoch 9/100
804/804 [=======

804/804 [==============================] - 1s 784us/step - loss: 0.5310 - accuracy: 0.7414
Epoch 63/100
804/804 [==============================] - 1s 785us/step - loss: 0.5313 - accuracy: 0.7410
Epoch 64/100
804/804 [==============================] - 1s 773us/step - loss: 0.5312 - accuracy: 0.7406
Epoch 65/100
796/804 [============================>.] - ETA: 0s - loss: 0.5308 - accuracy: 0.7408
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 775us/step - loss: 0.5313 - accuracy: 0.7407
Epoch 66/100
804/804 [==============================] - 1s 773us/step - loss: 0.5313 - accuracy: 0.7418
Epoch 67/100
804/804 [==============================] - 1s 882us/step - loss: 0.5314 - accuracy: 0.7401
Epoch 68/100
804/804 [==============================] - 1s 963us/step - loss: 0.5307 - accuracy: 0.7404
Epoch 69/100
804/804 [==============================] - 1s 784us/step - loss: 0.5309 - accuracy: 0.7406
Epoch 70/100
786/804 [==========

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5839 - accuracy: 0.7311 - 214ms/epoch - 798us/step
Loss: 0.583885669708252, Accuracy: 0.7310787439346313


In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 70
hidden_nodes_layer3 = 40

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 90)                4140      
                                                                 
 dense_14 (Dense)            (None, 70)                6370      
                                                                 
 dense_15 (Dense)            (None, 40)                2840      
                                                                 
 dense_16 (Dense)            (None, 1)                 41        
                                                                 
Total params: 13,391
Trainable params: 13,391
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [37]:
# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 814us/step - loss: 0.5656 - accuracy: 0.7226
Epoch 2/100
804/804 [==============================] - 1s 772us/step - loss: 0.5551 - accuracy: 0.7274
Epoch 3/100
804/804 [==============================] - 1s 746us/step - loss: 0.5527 - accuracy: 0.7291
Epoch 4/100
804/804 [==============================] - 1s 815us/step - loss: 0.5513 - accuracy: 0.7308
Epoch 5/100
403/804 [==============>...............] - ETA: 0s - loss: 0.5520 - accuracy: 0.7268
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 837us/step - loss: 0.5496 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 1s 747us/step - loss: 0.5486 - accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 1s 739us/step - loss: 0.5481 - accuracy: 0.7339
Epoch 8/100
804/804 [==============================] - 1s 919us/step - loss: 0.5467 - accuracy: 0.7327
Epoch 9/100
804/804 [=======

804/804 [==============================] - 1s 878us/step - loss: 0.5336 - accuracy: 0.7399
Epoch 63/100
804/804 [==============================] - 1s 853us/step - loss: 0.5334 - accuracy: 0.7392
Epoch 64/100
804/804 [==============================] - 1s 853us/step - loss: 0.5335 - accuracy: 0.7402
Epoch 65/100
417/804 [==============>...............] - ETA: 0s - loss: 0.5325 - accuracy: 0.7391
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 894us/step - loss: 0.5336 - accuracy: 0.7396
Epoch 66/100
804/804 [==============================] - 1s 882us/step - loss: 0.5333 - accuracy: 0.7399
Epoch 67/100
804/804 [==============================] - 1s 868us/step - loss: 0.5330 - accuracy: 0.7403
Epoch 68/100
804/804 [==============================] - 1s 866us/step - loss: 0.5334 - accuracy: 0.7399
Epoch 69/100
804/804 [==============================] - 1s 843us/step - loss: 0.5327 - accuracy: 0.7397
Epoch 70/100
402/804 [==========

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5740 - accuracy: 0.7303 - 200ms/epoch - 748us/step
Loss: 0.5740358233451843, Accuracy: 0.7302623987197876


In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 80)                3680      
                                                                 
 dense_18 (Dense)            (None, 50)                4050      
                                                                 
 dense_19 (Dense)            (None, 30)                1530      
                                                                 
 dense_20 (Dense)            (None, 1)                 31        
                                                                 
Total params: 9,291
Trainable params: 9,291
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [42]:
# Train the model
fit_model = nn5.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 716us/step - loss: 0.5666 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 1s 737us/step - loss: 0.5540 - accuracy: 0.7263
Epoch 3/100
804/804 [==============================] - 1s 712us/step - loss: 0.5524 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 1s 705us/step - loss: 0.5506 - accuracy: 0.7301
Epoch 5/100
784/804 [============================>.] - ETA: 0s - loss: 0.5488 - accuracy: 0.7297
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 743us/step - loss: 0.5494 - accuracy: 0.7293
Epoch 6/100
804/804 [==============================] - 1s 767us/step - loss: 0.5493 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 1s 773us/step - loss: 0.5480 - accuracy: 0.7319
Epoch 8/100
804/804 [==============================] - 1s 726us/step - loss: 0.5472 - accuracy: 0.7324
Epoch 9/100
804/804 [=======

804/804 [==============================] - 1s 734us/step - loss: 0.5341 - accuracy: 0.7391
Epoch 63/100
804/804 [==============================] - 1s 723us/step - loss: 0.5335 - accuracy: 0.7392
Epoch 64/100
804/804 [==============================] - 1s 698us/step - loss: 0.5338 - accuracy: 0.7392
Epoch 65/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5316 - accuracy: 0.7413
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 734us/step - loss: 0.5337 - accuracy: 0.7398
Epoch 66/100
804/804 [==============================] - 1s 711us/step - loss: 0.5334 - accuracy: 0.7390
Epoch 67/100
804/804 [==============================] - 1s 711us/step - loss: 0.5336 - accuracy: 0.7395
Epoch 68/100
804/804 [==============================] - 1s 709us/step - loss: 0.5337 - accuracy: 0.7395
Epoch 69/100
804/804 [==============================] - 1s 715us/step - loss: 0.5330 - accuracy: 0.7393
Epoch 70/100
773/804 [==========

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5717 - accuracy: 0.7315 - 190ms/epoch - 710us/step
Loss: 0.5716536641120911, Accuracy: 0.7315452098846436


In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 70
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 30

nn6 = tf.keras.models.Sequential()

# First hidden layer
nn6.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="LeakyReLU")
)

# Second hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn6.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn6.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 70)                3220      
                                                                 
 dense_24 (Dense)            (None, 40)                2840      
                                                                 
 dense_25 (Dense)            (None, 30)                1230      
                                                                 
 dense_26 (Dense)            (None, 1)                 31        
                                                                 
Total params: 7,321
Trainable params: 7,321
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile the model
nn6.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [50]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [51]:
# Train the model
fit_model = nn6.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 1s 742us/step - loss: 0.5688 - accuracy: 0.7194
Epoch 2/200
804/804 [==============================] - 1s 716us/step - loss: 0.5539 - accuracy: 0.7276
Epoch 3/200
804/804 [==============================] - 1s 711us/step - loss: 0.5518 - accuracy: 0.7286
Epoch 4/200
804/804 [==============================] - 1s 744us/step - loss: 0.5509 - accuracy: 0.7292
Epoch 5/200
788/804 [============================>.] - ETA: 0s - loss: 0.5506 - accuracy: 0.7308
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 727us/step - loss: 0.5502 - accuracy: 0.7312
Epoch 6/200
804/804 [==============================] - 1s 751us/step - loss: 0.5484 - accuracy: 0.7299
Epoch 7/200
804/804 [==============================] - 1s 763us/step - loss: 0.5483 - accuracy: 0.7310
Epoch 8/200
804/804 [==============================] - 1s 800us/step - loss: 0.5476 - accuracy: 0.7322
Epoch 9/200
804/804 [=======

804/804 [==============================] - 1s 748us/step - loss: 0.5376 - accuracy: 0.7376
Epoch 63/200
804/804 [==============================] - 1s 715us/step - loss: 0.5367 - accuracy: 0.7384
Epoch 64/200
804/804 [==============================] - 1s 713us/step - loss: 0.5370 - accuracy: 0.7379
Epoch 65/200
785/804 [============================>.] - ETA: 0s - loss: 0.5362 - accuracy: 0.7389
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 733us/step - loss: 0.5367 - accuracy: 0.7390
Epoch 66/200
804/804 [==============================] - 1s 709us/step - loss: 0.5370 - accuracy: 0.7379
Epoch 67/200
804/804 [==============================] - 1s 715us/step - loss: 0.5368 - accuracy: 0.7380
Epoch 68/200
804/804 [==============================] - 1s 719us/step - loss: 0.5366 - accuracy: 0.7384
Epoch 69/200
804/804 [==============================] - 1s 778us/step - loss: 0.5370 - accuracy: 0.7388
Epoch 70/200
774/804 [==========

804/804 [==============================] - 1s 776us/step - loss: 0.5345 - accuracy: 0.7397
Epoch 124/200
804/804 [==============================] - 1s 738us/step - loss: 0.5337 - accuracy: 0.7390
Epoch 125/200
786/804 [============================>.] - ETA: 0s - loss: 0.5344 - accuracy: 0.7387
Epoch 125: saving model to checkpoints_optimized\weights.125.hdf5
804/804 [==============================] - 1s 786us/step - loss: 0.5339 - accuracy: 0.7393
Epoch 126/200
804/804 [==============================] - 1s 777us/step - loss: 0.5339 - accuracy: 0.7390
Epoch 127/200
804/804 [==============================] - 1s 786us/step - loss: 0.5337 - accuracy: 0.7392
Epoch 128/200
804/804 [==============================] - 1s 838us/step - loss: 0.5333 - accuracy: 0.7399
Epoch 129/200
804/804 [==============================] - 1s 733us/step - loss: 0.5338 - accuracy: 0.7397
Epoch 130/200
750/804 [==========================>...] - ETA: 0s - loss: 0.5335 - accuracy: 0.7400
Epoch 130: saving model to ch

804/804 [==============================] - 1s 789us/step - loss: 0.5324 - accuracy: 0.7402
Epoch 184/200
804/804 [==============================] - 1s 802us/step - loss: 0.5333 - accuracy: 0.7395
Epoch 185/200
749/804 [==========================>...] - ETA: 0s - loss: 0.5328 - accuracy: 0.7395
Epoch 185: saving model to checkpoints_optimized\weights.185.hdf5
804/804 [==============================] - 1s 752us/step - loss: 0.5322 - accuracy: 0.7397
Epoch 186/200
804/804 [==============================] - 1s 737us/step - loss: 0.5332 - accuracy: 0.7400
Epoch 187/200
804/804 [==============================] - 1s 751us/step - loss: 0.5333 - accuracy: 0.7396
Epoch 188/200
804/804 [==============================] - 1s 791us/step - loss: 0.5324 - accuracy: 0.7396
Epoch 189/200
804/804 [==============================] - 1s 802us/step - loss: 0.5325 - accuracy: 0.7399
Epoch 190/200
802/804 [============================>.] - ETA: 0s - loss: 0.5326 - accuracy: 0.7400
Epoch 190: saving model to ch

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn6.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5701 - accuracy: 0.7310 - 193ms/epoch - 721us/step
Loss: 0.5700847506523132, Accuracy: 0.7309620976448059


In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn7 = tf.keras.models.Sequential()

# First hidden layer
nn7.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn7.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn7.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 80)                3680      
                                                                 
 dense_28 (Dense)            (None, 50)                4050      
                                                                 
 dense_29 (Dense)            (None, 30)                1530      
                                                                 
 dense_30 (Dense)            (None, 1)                 31        
                                                                 
Total params: 9,291
Trainable params: 9,291
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Compile the model
nn7.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [57]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [58]:
# Train the model
fit_model = nn7.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 753us/step - loss: 0.5680 - accuracy: 0.7203
Epoch 2/150
804/804 [==============================] - 1s 737us/step - loss: 0.5561 - accuracy: 0.7278
Epoch 3/150
804/804 [==============================] - 1s 709us/step - loss: 0.5524 - accuracy: 0.7291
Epoch 4/150
804/804 [==============================] - 1s 700us/step - loss: 0.5507 - accuracy: 0.7302
Epoch 5/150
788/804 [============================>.] - ETA: 0s - loss: 0.5498 - accuracy: 0.7305
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 719us/step - loss: 0.5496 - accuracy: 0.7306
Epoch 6/150
804/804 [==============================] - 1s 738us/step - loss: 0.5494 - accuracy: 0.7320
Epoch 7/150
804/804 [==============================] - 1s 811us/step - loss: 0.5479 - accuracy: 0.7306
Epoch 8/150
804/804 [==============================] - 1s 738us/step - loss: 0.5477 - accuracy: 0.7311
Epoch 9/150
804/804 [=======

804/804 [==============================] - 1s 757us/step - loss: 0.5324 - accuracy: 0.7402
Epoch 63/150
804/804 [==============================] - 1s 726us/step - loss: 0.5322 - accuracy: 0.7400
Epoch 64/150
804/804 [==============================] - 1s 741us/step - loss: 0.5322 - accuracy: 0.7395
Epoch 65/150
754/804 [===========================>..] - ETA: 0s - loss: 0.5321 - accuracy: 0.7401
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 818us/step - loss: 0.5321 - accuracy: 0.7400
Epoch 66/150
804/804 [==============================] - 1s 713us/step - loss: 0.5318 - accuracy: 0.7404
Epoch 67/150
804/804 [==============================] - 1s 734us/step - loss: 0.5320 - accuracy: 0.7403
Epoch 68/150
804/804 [==============================] - 1s 731us/step - loss: 0.5316 - accuracy: 0.7404
Epoch 69/150
804/804 [==============================] - 1s 719us/step - loss: 0.5316 - accuracy: 0.7402
Epoch 70/150
763/804 [==========

804/804 [==============================] - 1s 825us/step - loss: 0.5287 - accuracy: 0.7414
Epoch 124/150
804/804 [==============================] - 1s 754us/step - loss: 0.5305 - accuracy: 0.7413
Epoch 125/150
793/804 [============================>.] - ETA: 0s - loss: 0.5287 - accuracy: 0.7418
Epoch 125: saving model to checkpoints_optimized\weights.125.hdf5
804/804 [==============================] - 1s 785us/step - loss: 0.5292 - accuracy: 0.7415
Epoch 126/150
804/804 [==============================] - 1s 750us/step - loss: 0.5292 - accuracy: 0.7413
Epoch 127/150
804/804 [==============================] - 1s 767us/step - loss: 0.5292 - accuracy: 0.7410
Epoch 128/150
804/804 [==============================] - 1s 872us/step - loss: 0.5293 - accuracy: 0.7412
Epoch 129/150
804/804 [==============================] - 1s 775us/step - loss: 0.5290 - accuracy: 0.7412
Epoch 130/150
748/804 [==========================>...] - ETA: 0s - loss: 0.5292 - accuracy: 0.7410
Epoch 130: saving model to ch

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn7.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5774 - accuracy: 0.7275 - 208ms/epoch - 775us/step
Loss: 0.5774033665657043, Accuracy: 0.7274635434150696


In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 85
hidden_nodes_layer2 = 55
hidden_nodes_layer3 = 35
hidden_nodes_layer4 = 25

nn9 = tf.keras.models.Sequential()

# First hidden layer
nn9.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# 4th hidden layer
nn9.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn9.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn9.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 85)                3910      
                                                                 
 dense_37 (Dense)            (None, 55)                4730      
                                                                 
 dense_38 (Dense)            (None, 35)                1960      
                                                                 
 dense_39 (Dense)            (None, 25)                900       
                                                                 
 dense_40 (Dense)            (None, 1)                 26        
                                                                 
Total params: 11,526
Trainable params: 11,526
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Compile the model
nn9.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [67]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [68]:
# Train the model
fit_model = nn9.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 804us/step - loss: 0.5663 - accuracy: 0.7207
Epoch 2/150
804/804 [==============================] - 1s 864us/step - loss: 0.5543 - accuracy: 0.7294
Epoch 3/150
804/804 [==============================] - 1s 768us/step - loss: 0.5517 - accuracy: 0.7286
Epoch 4/150
804/804 [==============================] - 1s 772us/step - loss: 0.5497 - accuracy: 0.7315
Epoch 5/150
794/804 [============================>.] - ETA: 0s - loss: 0.5486 - accuracy: 0.7311
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 791us/step - loss: 0.5487 - accuracy: 0.7309
Epoch 6/150
804/804 [==============================] - 1s 822us/step - loss: 0.5478 - accuracy: 0.7317
Epoch 7/150
804/804 [==============================] - 1s 783us/step - loss: 0.5468 - accuracy: 0.7323
Epoch 8/150
804/804 [==============================] - 1s 832us/step - loss: 0.5465 - accuracy: 0.7337
Epoch 9/150
804/804 [=======

804/804 [==============================] - 1s 800us/step - loss: 0.5341 - accuracy: 0.7392
Epoch 63/150
804/804 [==============================] - 1s 778us/step - loss: 0.5336 - accuracy: 0.7389
Epoch 64/150
804/804 [==============================] - 1s 768us/step - loss: 0.5336 - accuracy: 0.7393
Epoch 65/150
793/804 [============================>.] - ETA: 0s - loss: 0.5335 - accuracy: 0.7390
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 786us/step - loss: 0.5333 - accuracy: 0.7392
Epoch 66/150
804/804 [==============================] - 1s 934us/step - loss: 0.5330 - accuracy: 0.7398
Epoch 67/150
804/804 [==============================] - 1s 777us/step - loss: 0.5332 - accuracy: 0.7391
Epoch 68/150
804/804 [==============================] - 1s 778us/step - loss: 0.5331 - accuracy: 0.7400
Epoch 69/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5331 - accuracy: 0.7396
Epoch 70/150
771/804 [============

804/804 [==============================] - 1s 851us/step - loss: 0.5298 - accuracy: 0.7413
Epoch 124/150
804/804 [==============================] - 1s 832us/step - loss: 0.5309 - accuracy: 0.7405
Epoch 125/150
786/804 [============================>.] - ETA: 0s - loss: 0.5295 - accuracy: 0.7420
Epoch 125: saving model to checkpoints_optimized\weights.125.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5302 - accuracy: 0.7414
Epoch 126/150
804/804 [==============================] - 1s 850us/step - loss: 0.5303 - accuracy: 0.7408
Epoch 127/150
804/804 [==============================] - 1s 833us/step - loss: 0.5305 - accuracy: 0.7411
Epoch 128/150
804/804 [==============================] - 1s 833us/step - loss: 0.5301 - accuracy: 0.7408
Epoch 129/150
804/804 [==============================] - 1s 853us/step - loss: 0.5312 - accuracy: 0.7412
Epoch 130/150
801/804 [============================>.] - ETA: 0s - loss: 0.5299 - accuracy: 0.7410
Epoch 130: saving model to chec

In [70]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn9.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5791 - accuracy: 0.7324 - 199ms/epoch - 744us/step
Loss: 0.579121470451355, Accuracy: 0.7323614954948425


In [174]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")


# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"],1)
application_df


# Determine which values to replace if counts are less than 777?
replace_application = list(application_type[application_type < 777].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
    
# Determine which values to replace if counts are less than 1900?
replace_class = list(classification[classification < 1700].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
    
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[app_cat].nunique()

C:\Users\rghaf\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


APPLICATION_TYPE          6
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [175]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)


# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(app_cat,1)
application_df.head()

C:\Users\rghaf\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\rghaf\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [176]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [177]:
len(X_train_scaled[0])

40

In [262]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 85
hidden_nodes_layer2 = 55
hidden_nodes_layer3 = 35
hidden_nodes_layer4 = 25
#hidden_nodes_layer5 = 20

nn39 =  tf.keras.models.Sequential()

# First hidden layer
nn39.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn39.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn39.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# 4th hidden layer
nn39.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# 5th hidden layer
#nn37.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn39.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn39.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_173 (Dense)           (None, 85)                3485      
                                                                 
 dense_174 (Dense)           (None, 55)                4730      
                                                                 
 dense_175 (Dense)           (None, 35)                1960      
                                                                 
 dense_176 (Dense)           (None, 25)                900       
                                                                 
 dense_177 (Dense)           (None, 1)                 26        
                                                                 
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [263]:
# Compile the model
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import keras
optimizer = tf.keras.optimizers.Adam(0.001)
#optimizer.learning_rate.assign(0.0009)
nn39.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [264]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized/",exist_ok=True)
checkpoint_path = "checkpoints_optimized/weights.{epoch:02d}.hdf5"

batch_size = 804
# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [265]:
# Train the model
fit_model = nn39.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5731 - accuracy: 0.7198
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5578 - accuracy: 0.7288
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5558 - accuracy: 0.7294
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7311
Epoch 5/150
797/804 [============================>.] - ETA: 0s - loss: 0.5537 - accuracy: 0.7308
Epoch 5: saving model to checkpoints_optimized\weights.05.hdf5
804/804 [==============================] - 1s 978us/step - loss: 0.5533 - accuracy: 0.7312
Epoch 6/150
804/804 [==============================] - 1s 965us/step - loss: 0.5524 - accuracy: 0.7315
Epoch 7/150
804/804 [==============================] - 1s 915us/step - loss: 0.5520 - accuracy: 0.7320
Epoch 8/150
804/804 [==============================] - 1s 920us/step - loss: 0.5512 - accuracy: 0.7315
Epoch 9/150
804/804 [===============

804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7378
Epoch 64/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7386
Epoch 65/150
796/804 [============================>.] - ETA: 0s - loss: 0.5394 - accuracy: 0.7376
Epoch 65: saving model to checkpoints_optimized\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7377
Epoch 66/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7381
Epoch 67/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5401 - accuracy: 0.7378
Epoch 68/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7380
Epoch 69/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7376
Epoch 70/150
800/804 [============================>.] - ETA: 0s - loss: 0.5395 - accuracy: 0.7382
Epoch 70: saving model to checkpoints_optimized\weig

788/804 [============================>.] - ETA: 0s - loss: 0.5363 - accuracy: 0.7395
Epoch 125: saving model to checkpoints_optimized\weights.125.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5363 - accuracy: 0.7395
Epoch 126/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7382
Epoch 127/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7396
Epoch 128/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7392
Epoch 129/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7395
Epoch 130/150
795/804 [============================>.] - ETA: 0s - loss: 0.5371 - accuracy: 0.7396
Epoch 130: saving model to checkpoints_optimized\weights.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7394
Epoch 131/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy

In [267]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn39.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5878 - accuracy: 0.7312 - 314ms/epoch - 1ms/step
Loss: 0.5877928137779236, Accuracy: 0.731195330619812


In [268]:
# Export our model to HDF5 file
nn39.save("AlphabetSoupCharity_Optimization.h5")